In [5]:
import pandas as pd
import numpy as np
import FinanceDataReader as fdr
import matplotlib.pyplot as plt

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, Bidirectional

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

import xgboost as xgb

In [6]:
def MinMaxScaler(data):
    numerator = data - np.min(data, 0)
    denominator = np.max(data, 0) - np.min(data, 0)
    return numerator / (denominator + 1e-7)

In [71]:
def BLSTM(finnum):    
    df = fdr.DataReader(f'{finnum}', '2018-01-01', '2023-01-01')
    dfx = df[['Open','High','Low','Volume', 'Close']]
    dfx = MinMaxScaler(dfx)
    dfy = dfx[['Close']]
    dfx = dfx[['Open','High','Low','Volume']]
    dfx=dfx.reset_index()
    dfx=dfx.set_index(keys='Date')


    X = dfx.values.tolist()
    y = dfy.values.tolist()

    #며칠데이터로 예측
    window_size = 10

    data_X = []
    data_y = []
    for i in range(len(y) - window_size):
        _X = X[i : i + window_size] 
        _y = y[i + window_size]     
        data_X.append(_X)
        data_y.append(_y)


    n=len(data_y)

    train_X = np.array(data_X[0:int(n*0.7)])
    train_y = np.array(data_y[0:int(n*0.7)])

    val_X = np.array(data_X[int(n*0.7):int(n*0.9)])
    val_y = np.array(data_y[int(n*0.7):int(n*0.9)])

    test_X = np.array(data_X[int(n*0.9):])
    test_y = np.array(data_y[int(n*0.9):])

    validation_data = (val_X,val_y)
    
    
    model2 = Sequential()


    forward_layer = LSTM(64, return_sequences=True)
    backward_layer = LSTM(64, activation='relu', return_sequences=True,
                          go_backwards=True)

    model2.add(Bidirectional(forward_layer, backward_layer=backward_layer,
                            input_shape=(10, 4)))

    model2.add(Dropout(0.2))

    model2.add(Bidirectional(LSTM(64)))

    model2.add(Dropout(0.2))

    model2.add(Dense(units=1))


    model2.summary()

    model2.compile(optimizer='adam', 
                  loss='mean_squared_error')
    model2.fit(train_X, 
              train_y, 
              epochs=100, 
              batch_size=30,
              validation_data=validation_data
             )

    pred_y = model2.predict(test_X)
    
    
    RMSE_predict=np.sqrt(mean_squared_error(test_y, pred_y))
    f2_y_predict = r2_score(test_y, pred_y)
    
    return RMSE_predict,f2_y_predict

In [72]:
a,b,c=BLSTM('035420')

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_10 (Bidirecti  (None, 10, 128)          35328     
 onal)                                                           
                                                                 
 dropout_10 (Dropout)        (None, 10, 128)           0         
                                                                 
 bidirectional_11 (Bidirecti  (None, 128)              98816     
 onal)                                                           
                                                                 
 dropout_11 (Dropout)        (None, 128)               0         
                                                                 
 dense_5 (Dense)             (None, 1)                 129       
                                                                 
Total params: 134,273
Trainable params: 134,273
Non-tr

KeyboardInterrupt: 

In [18]:
a

0.0008701217593625188

In [19]:
b

0.02949782633780208

In [20]:
c

0.9113183897804472

In [73]:
kospi50 = pd.read_csv('./Kospi50.csv', encoding='cp949')

In [74]:
kospi50

,종목코드,종목명
0,5930,삼성전자
1,373220,LG에너지솔루션
2,660,SK하이닉스
3,207940,삼성바이오로직스
4,5490,POSCO홀딩스
5,6400,삼성SDI
6,51910,LG화학
7,5380,현대차
8,35420,NAVER
9,3670,포스코퓨처엠


In [75]:
kospi50['종목코드']=kospi50['종목코드'].apply(lambda x : str(x).zfill(6))

In [76]:
kospi50['RMSE']=np.nan
kospi50['R2']=np.nan
kospi50

,종목코드,종목명,RMSE,R2
0,005930,삼성전자,NaN,NaN
1,373220,LG에너지솔루션,NaN,NaN
2,000660,SK하이닉스,NaN,NaN
3,207940,삼성바이오로직스,NaN,NaN
4,005490,POSCO홀딩스,NaN,NaN
5,006400,삼성SDI,NaN,NaN
6,051910,LG화학,NaN,NaN
7,005380,현대차,NaN,NaN
8,035420,NAVER,NaN,NaN
9,003670,포스코퓨처엠,NaN,NaN


In [ ]:
for i in range(0,50):
    RMSE,R2=BLSTM(kospi50.iloc[i,0])
    kospi50.iloc[i,2]=RMSE
    kospi50.iloc[i,3]=R2
    print(kospi50.iloc[i,1])

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_14 (Bidirecti  (None, 10, 128)          35328     
 onal)                                                           
                                                                 
 dropout_14 (Dropout)        (None, 10, 128)           0         
                                                                 
 bidirectional_15 (Bidirecti  (None, 128)              98816     
 onal)                                                           
                                                                 
 dropout_15 (Dropout)        (None, 128)               0         
                                                                 
 dense_7 (Dense)             (None, 1)                 129       
                                                                 
Total params: 134,273
Trainable params: 134,273
Non-tr

29/29 [==============================] - 0s 17ms/step - loss: 0.0012 - val_loss: 0.0012
Epoch 71/100
29/29 [==============================] - 0s 15ms/step - loss: 0.0013 - val_loss: 0.0021
Epoch 72/100
29/29 [==============================] - 0s 16ms/step - loss: 0.0013 - val_loss: 9.8586e-04
Epoch 73/100
29/29 [==============================] - 0s 16ms/step - loss: 0.0012 - val_loss: 0.0012
Epoch 74/100
29/29 [==============================] - 0s 16ms/step - loss: 0.0012 - val_loss: 0.0038
Epoch 75/100
29/29 [==============================] - 0s 16ms/step - loss: 0.0014 - val_loss: 0.0012
Epoch 76/100
29/29 [==============================] - 0s 15ms/step - loss: 0.0013 - val_loss: 0.0016
Epoch 77/100
29/29 [==============================] - 0s 15ms/step - loss: 0.0012 - val_loss: 0.0021
Epoch 78/100
29/29 [==============================] - 0s 15ms/step - loss: 0.0013 - val_loss: 0.0023
Epoch 79/100
29/29 [==============================] - 0s 15ms/step - loss: 0.0011 - val_loss: 0.0015

6/6 [==============================] - 0s 20ms/step - loss: 0.0027 - val_loss: 0.0120
Epoch 40/100
6/6 [==============================] - 0s 21ms/step - loss: 0.0033 - val_loss: 0.0093
Epoch 41/100
6/6 [==============================] - 0s 21ms/step - loss: 0.0028 - val_loss: 0.0093
Epoch 42/100
6/6 [==============================] - 0s 19ms/step - loss: 0.0029 - val_loss: 0.0054
Epoch 43/100
6/6 [==============================] - 0s 20ms/step - loss: 0.0033 - val_loss: 0.0201
Epoch 44/100
6/6 [==============================] - 0s 19ms/step - loss: 0.0030 - val_loss: 0.0094
Epoch 45/100
6/6 [==============================] - 0s 20ms/step - loss: 0.0032 - val_loss: 0.0131
Epoch 46/100
6/6 [==============================] - 0s 19ms/step - loss: 0.0031 - val_loss: 0.0121
Epoch 47/100
6/6 [==============================] - 0s 20ms/step - loss: 0.0030 - val_loss: 0.0145
Epoch 48/100
6/6 [==============================] - 0s 20ms/step - loss: 0.0028 - val_loss: 0.0185
Epoch 49/100
6/6 [=====

Epoch 9/100
29/29 [==============================] - 0s 15ms/step - loss: 0.0024 - val_loss: 0.0020
Epoch 10/100
29/29 [==============================] - 0s 15ms/step - loss: 0.0024 - val_loss: 0.0020
Epoch 11/100
29/29 [==============================] - 0s 15ms/step - loss: 0.0024 - val_loss: 0.0021
Epoch 12/100
29/29 [==============================] - 0s 15ms/step - loss: 0.0023 - val_loss: 0.0018
Epoch 13/100
29/29 [==============================] - 0s 15ms/step - loss: 0.0024 - val_loss: 0.0024
Epoch 14/100
29/29 [==============================] - 0s 15ms/step - loss: 0.0022 - val_loss: 0.0021
Epoch 15/100
29/29 [==============================] - 0s 15ms/step - loss: 0.0020 - val_loss: 0.0027
Epoch 16/100
29/29 [==============================] - 0s 15ms/step - loss: 0.0022 - val_loss: 0.0017
Epoch 17/100
29/29 [==============================] - 0s 14ms/step - loss: 0.0021 - val_loss: 0.0017
Epoch 18/100
29/29 [==============================] - 0s 15ms/step - loss: 0.0019 - val_loss

29/29 [==============================] - 0s 16ms/step - loss: 0.0011 - val_loss: 0.0015
Epoch 91/100
29/29 [==============================] - 0s 15ms/step - loss: 0.0012 - val_loss: 0.0012
Epoch 92/100
29/29 [==============================] - 0s 15ms/step - loss: 0.0012 - val_loss: 0.0020
Epoch 93/100
29/29 [==============================] - 0s 15ms/step - loss: 0.0013 - val_loss: 0.0036
Epoch 94/100
29/29 [==============================] - 0s 15ms/step - loss: 0.0012 - val_loss: 0.0014
Epoch 95/100
29/29 [==============================] - 0s 15ms/step - loss: 0.0012 - val_loss: 0.0013
Epoch 96/100
29/29 [==============================] - 0s 15ms/step - loss: 0.0011 - val_loss: 0.0011
Epoch 97/100
29/29 [==============================] - 0s 16ms/step - loss: 0.0011 - val_loss: 0.0023
Epoch 98/100
29/29 [==============================] - 0s 16ms/step - loss: 0.0010 - val_loss: 0.0020
Epoch 99/100
29/29 [==============================] - 0s 15ms/step - loss: 0.0012 - val_loss: 0.0013
Epo

...

 dropout_100 (Dropout)       (None, 10, 128)           0         
                                                                 
 bidirectional_101 (Bidirect  (None, 128)              98816     
 ional)                                                          
                                                                 
 dropout_101 (Dropout)       (None, 128)               0         
                                                                 
 dense_50 (Dense)            (None, 1)                 129       
                                                                 
Total params: 134,273
Trainable params: 134,273
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
29/29 [==============================] - 8s 75ms/step - loss: 0.0304 - val_loss: 0.0032
Epoch 2/100
29/29 [==============================] - 1s 19ms/step - loss: 0.0047 - val_loss: 0.0021
Epoch 3/100
29/29 [==============================] - 1s 19ms/step - 

29/29 [==============================] - 1s 30ms/step - loss: 0.0016 - val_loss: 9.2112e-04
Epoch 75/100
29/29 [==============================] - 1s 39ms/step - loss: 0.0016 - val_loss: 9.5284e-04
Epoch 76/100
29/29 [==============================] - 1s 41ms/step - loss: 0.0014 - val_loss: 0.0011
Epoch 77/100
29/29 [==============================] - 1s 25ms/step - loss: 0.0016 - val_loss: 0.0012
Epoch 78/100
29/29 [==============================] - 1s 29ms/step - loss: 0.0014 - val_loss: 0.0012
Epoch 79/100
29/29 [==============================] - 1s 25ms/step - loss: 0.0015 - val_loss: 9.4949e-04
Epoch 80/100
29/29 [==============================] - 1s 22ms/step - loss: 0.0014 - val_loss: 9.0883e-04
Epoch 81/100
29/29 [==============================] - 1s 23ms/step - loss: 0.0014 - val_loss: 8.9974e-04
Epoch 82/100
29/29 [==============================] - 1s 27ms/step - loss: 0.0013 - val_loss: 9.2369e-04
Epoch 83/100
29/29 [==============================] - 1s 31ms/step - loss: 0.001

7/7 [==============================] - 0s 21ms/step - loss: 0.0034 - val_loss: 0.0032
Epoch 44/100
7/7 [==============================] - 0s 44ms/step - loss: 0.0040 - val_loss: 0.0031
Epoch 45/100
7/7 [==============================] - 0s 48ms/step - loss: 0.0036 - val_loss: 0.0031
Epoch 46/100
7/7 [==============================] - 0s 25ms/step - loss: 0.0038 - val_loss: 0.0030
Epoch 47/100
7/7 [==============================] - 0s 29ms/step - loss: 0.0040 - val_loss: 0.0029
Epoch 48/100
7/7 [==============================] - 0s 65ms/step - loss: 0.0033 - val_loss: 0.0028
Epoch 49/100
7/7 [==============================] - 0s 31ms/step - loss: 0.0040 - val_loss: 0.0029
Epoch 50/100
7/7 [==============================] - 0s 28ms/step - loss: 0.0030 - val_loss: 0.0031
Epoch 51/100
7/7 [==============================] - 0s 26ms/step - loss: 0.0033 - val_loss: 0.0031
Epoch 52/100
7/7 [==============================] - 0s 21ms/step - loss: 0.0035 - val_loss: 0.0030
Epoch 53/100
7/7 [=====

29/29 [==============================] - 1s 18ms/step - loss: 0.0017 - val_loss: 4.9643e-04
Epoch 13/100
29/29 [==============================] - 1s 26ms/step - loss: 0.0018 - val_loss: 4.9619e-04
Epoch 14/100
29/29 [==============================] - 1s 21ms/step - loss: 0.0018 - val_loss: 5.4513e-04
Epoch 15/100
29/29 [==============================] - 1s 24ms/step - loss: 0.0017 - val_loss: 4.5083e-04
Epoch 16/100
29/29 [==============================] - 1s 23ms/step - loss: 0.0017 - val_loss: 4.4933e-04
Epoch 17/100
29/29 [==============================] - 1s 23ms/step - loss: 0.0019 - val_loss: 5.1852e-04
Epoch 18/100
29/29 [==============================] - 1s 19ms/step - loss: 0.0019 - val_loss: 4.2349e-04
Epoch 19/100
29/29 [==============================] - 1s 22ms/step - loss: 0.0014 - val_loss: 4.8440e-04
Epoch 20/100
29/29 [==============================] - 1s 24ms/step - loss: 0.0016 - val_loss: 5.4955e-04
Epoch 21/100
29/29 [==============================] - 1s 28ms/step -

Epoch 90/100
29/29 [==============================] - 1s 21ms/step - loss: 0.0010 - val_loss: 2.9856e-04
Epoch 91/100
29/29 [==============================] - 1s 21ms/step - loss: 0.0010 - val_loss: 2.5461e-04
Epoch 92/100
29/29 [==============================] - 1s 21ms/step - loss: 0.0011 - val_loss: 3.0903e-04
Epoch 93/100
29/29 [==============================] - 1s 22ms/step - loss: 0.0010 - val_loss: 3.1190e-04
Epoch 94/100
29/29 [==============================] - 1s 45ms/step - loss: 9.2769e-04 - val_loss: 3.0035e-04
Epoch 95/100
29/29 [==============================] - 1s 24ms/step - loss: 9.2740e-04 - val_loss: 2.8274e-04
Epoch 96/100
29/29 [==============================] - 1s 19ms/step - loss: 0.0011 - val_loss: 2.5129e-04
Epoch 97/100
29/29 [==============================] - 1s 19ms/step - loss: 0.0011 - val_loss: 2.8080e-04
Epoch 98/100
29/29 [==============================] - 1s 19ms/step - loss: 0.0010 - val_loss: 2.5969e-04
Epoch 99/100
29/29 [===========================

In [ ]:
import pickle

with open( "./kospi50BLSTM", "wb" ) as file:
    pickle.dump( kospi50, file)